In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='AppleGothic')
import warnings
warnings.filterwarnings("ignore")

In [2]:
세종시_버스인원 = pd.read_csv('./행정동_joined//1720_버스인원.csv')

In [3]:
#마지막 행 삭제
세종시_버스인원_총이용인원 = 세종시_버스인원.iloc[-1,:]
세종시_버스인원 = 세종시_버스인원.iloc[:-1,:]


In [4]:
#합계 행들 삭제
idx = 세종시_버스인원[세종시_버스인원['행정구역'] == "합계"].index
세종시_버스인원 = 세종시_버스인원.drop(idx)

In [5]:
# 부동산지표 및 전입전출자수 를 활용한 KMeans에서 얻은 라벨을 활용하여 Label을 붙여줍니다
#A군집 : 아름동, 연동면, 연서면, 금남면, 고운동, 도담동, 종촌동, 조치원읍, 연기면
#B군집 : 전동면, 부강면, 소정면, 전의면, 장군면
#C군집 : 소담동, 한솔동, 새롬동, 대평동, 보람동

In [6]:
#가람동 -> 힌솔동
#어진동 -> 도담동
#다정동, 나성동 -> 새롬동
#반곡동 -> 소담동
#반포면 -> 세종시 아님. 공주시 -> 삭제
세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace('가람동', '한솔동')
세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace('어진동', '도담동')
세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace('다정동', '새롬동')
세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace('나성동', '새롬동')
세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace('반곡동', '소담동')

### 해당 군집의 Label을 붙여줍니다

In [7]:
A = ['아름동', '연동면', '연서면', '금남면', '고운동', '도담동', '종촌동','조치원읍', '연기면']
B = ['전동면', '부강면', '소정면', '전의면', '장군면']
C = ['소담동', '한솔동', '새롬동', '대평동', '보람동']


for name in A:
    세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace( name, 'A')

for name in B:
    세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace( name, 'B')
    
for name in C:
    세종시_버스인원['행정구역'] = 세종시_버스인원.행정구역.str.replace( name, 'C')

#### 반포면은 세종시의 행정구역이 아니여서 제거했습니다

In [8]:
idx = 세종시_버스인원[세종시_버스인원['행정구역'] == "반포면"].index
세종시_버스인원 = 세종시_버스인원.drop(idx)

In [9]:
세종시_버스인원['연도'] = 세종시_버스인원.일자.str[0:4]

#### 수치화하기 위해 1000에 붙어있는 콤마를 제거해 주었습니다

In [10]:
세종시_버스인원['승차'] = 세종시_버스인원['승차'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
세종시_버스인원['하차'] = 세종시_버스인원['하차'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
세종시_버스인원['환승'] = 세종시_버스인원['환승'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

#### 결측값은 0을 의미하는 것이기 때문에 0으로 대체해 주었습니다

In [11]:
세종시_버스인원 = 세종시_버스인원.fillna(0)

In [12]:
세종시_버스인원['승차'] = 세종시_버스인원.승차.astype(int)
세종시_버스인원['하차'] = 세종시_버스인원.하차.astype(int)
세종시_버스인원['환승'] = 세종시_버스인원.환승.astype(int)

In [13]:
승차 = pd.pivot_table(세종시_버스인원, index = '연도', columns = '행정구역', values = '승차' , aggfunc = 'sum').reset_index()

하차 = pd.pivot_table(세종시_버스인원, index = '연도', columns = '행정구역', values = '하차' , aggfunc = 'sum').reset_index()

환승 = pd.pivot_table(세종시_버스인원, index = '연도', columns = '행정구역', values = '환승' , aggfunc = 'sum').reset_index()

In [14]:
군집별 = 승차+하차+환승
군집별['연도'] = 군집별.연도.str[0:4]

In [15]:
군집별.to_csv('./군집화된_외부데이터/세종시_군집별_연도별_버스이용객.csv',index = False)

## 군집별로 되어있는 데이터 합치기

In [16]:
cluster_A = pd.read_csv('./군집화된_외부데이터/A군집_면적_버스정류장.csv')
cluster_B = pd.read_csv('./군집화된_외부데이터/B군집_면적_버스정류장.csv')
cluster_C = pd.read_csv('./군집화된_외부데이터/C군집_면적_버스정류장.csv')
cluster_A['군집'] = 'A'
cluster_B['군집'] = 'B'
cluster_C['군집'] = 'C'
merged = pd.concat([cluster_A,cluster_B,cluster_C])
merged.to_csv('./군집화된_외부데이터/세종시_버스정류장_군집행정동추가.csv', index = False)